# Transition-based swapping arc-standard unlabeled dependency parser for Ukrainian

В цій роботі я спробував побудувати варіант transition-based алгоритму який вміє працювати з нетранзитивними деревами залежностей.

## Get the Data
Спочатку отримуюємо тренувальні та тестові данні.

In [3]:
!curl https://raw.githubusercontent.com/UniversalDependencies/UD_Ukrainian-IU/master/uk_iu-ud-train.conllu --output uk_iu-ud-train.conllu

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11.8M  100 11.8M    0     0  5046k      0  0:00:02  0:00:02 --:--:-- 5046k


In [19]:
!head -n 10 uk_iu-ud-train.conllu

# doc_title = «Я обізвуся до них…»
# newdoc id = 0000
# newpar id = 0001
# sent_id = 0002
# text = У домі римського патриція Руфіна була прегарна фреска, зображення Венери та Адоніса.
# translit = U domi rymśkoho patrycija Rufina bula preharna freska, zobraženńа Venery ta Adonisa.
1	У	у	ADP	Spsl	Case=Loc	2	case	2:case	Id=0003|LTranslit=u|Translit=U
2	домі	дім	NOUN	Ncmsln	Animacy=Inan|Case=Loc|Gender=Masc|Number=Sing	6	obl	6:obl	Id=0004|LTranslit=dim|Translit=domi
3	римського	римський	ADJ	Ao-msgf	Case=Gen|Gender=Masc|Number=Sing	4	amod	4:amod	Id=0005|LTranslit=rymśkyj|Translit=rymśkoho
4	патриція	патрицій	NOUN	Ncmsgy	Animacy=Anim|Case=Gen|Gender=Masc|Number=Sing	2	nmod	2:nmod	Id=0006|LTranslit=patrycij|Translit=patrycija


In [20]:
!curl https://raw.githubusercontent.com/UniversalDependencies/UD_Ukrainian-IU/master/uk_iu-ud-test.conllu --output uk_iu-ud-test.conllu

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2308k  100 2308k    0     0  1643k      0  0:00:01  0:00:01 --:--:-- 1643k


In [1]:
!head -n 10 uk_iu-ud-test.conllu

# doc_title = «Я обізвуся до них…»
# newdoc id = 0000
# newpar id = 01rc
# sent_id = 01rd
# text = Зречення культурної ідентичності – це втрата свободи й самовладності.
# translit = Zrečenńа kuľturnoji identyčnosti – ce vtrata svobody j samovladnosti.
1	Зречення	зречення	NOUN	Ncnsnn	Animacy=Inan|Case=Nom|Gender=Neut|Number=Sing	6	nsubj	6:nsubj	Id=01re|LTranslit=zrečenńа|Translit=Zrečenńа
2	культурної	культурний	ADJ	Afpfsgf	Case=Gen|Degree=Pos|Gender=Fem|Number=Sing	3	amod	3:amod	Id=01rf|LTranslit=kuľturnyj|Translit=kuľturnoji
3	ідентичності	ідентичність	NOUN	Ncfsgn	Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing	1	nmod	1:nmod	Id=01rg|LTranslit=identyčnisť|Translit=identyčnosti
4	–	–	PUNCT	U	PunctType=Dash	6	punct	6:punct	Id=01rh|LTranslit=–|Translit=–


In [2]:
import conllu

def read_treebank(filename):
    with open(filename) as f:
        return conllu.parse(f.read())

trees = read_treebank('uk_iu-ud-train.conllu')

In [4]:
len(trees)

5290

In [5]:
trees[0][2]

OrderedDict([('id', 3),
             ('form', 'римського'),
             ('lemma', 'римський'),
             ('upostag', 'ADJ'),
             ('xpostag', 'Ao-msgf'),
             ('feats',
              OrderedDict([('Case', 'Gen'),
                           ('Gender', 'Masc'),
                           ('Number', 'Sing')])),
             ('head', 4),
             ('deprel', 'amod'),
             ('deps', [('amod', 4)]),
             ('misc',
              OrderedDict([('Id', '0005'),
                           ('LTranslit', 'rymśkyj'),
                           ('Translit', 'rymśkoho')]))])

In [6]:
def print_rels(tree):
    for node in tree:
        head = node["head"]
        print("{} <-- {}".format(node["form"],
                             tree[head - 1]["form"]
                             if head > 0 else "root"))

In [7]:
print_rels(trees[0])

У <-- домі
домі <-- була
римського <-- патриція
патриція <-- домі
Руфіна <-- патриція
була <-- root
прегарна <-- фреска
фреска <-- була
, <-- зображення
зображення <-- фреска
Венери <-- зображення
та <-- Адоніса
Адоніса <-- Венери
. <-- була


В деяких деревах ми маємо незрозумілим чином заданого батька.

In [8]:
def is_broken(tree):
    for n in tree:
        if type(n["id"]) != int:
            return True
    return False    

broken_trees = list(filter(is_broken, trees))

len(broken_trees)

177

In [9]:
for i, n in enumerate(broken_trees[0]):
    print(i, n['id'], n['head'], n['form'])
    

0 1 5 Найперш
1 2 3 барикада
2 3 0 мала
3 4 3 би
4 5 3 перегородити
5 6 7 затишну
6 7 5 вітальню
7 8 9 косачівського
8 9 7 дому
9 10 24 ,
10 11 24 бо
11 12 14 ,
12 13 14 як
13 14 24 дізнаємося
14 15 17 з
15 16 17 академічних
16 17 14 пояснень
17 18 14 ,
18 19 24 Ольга
19 20 19 Драгоманова
20 21 22 -
21 22 20 Косач
22 23 24 «
23 24 3 стояла
24 25 29 на
25 26 28 ліберально
26 27 26 -
27 28 29 буржуазних
28 29 24 позиціях
29 30 24 »
30 31 34 ,
31 32 34 а
32 33 34 її
33 34 24 донька
34 (34, '.', 1) None стояла
35 35 37 –
36 36 37 на
37 37 34 марксистських
38 38 3 .


Такі дерева ми відфільтровуємо.

In [10]:
clean_trees = list(filter(lambda t: not is_broken(t), trees))

len(clean_trees)

5113

Далі знаходимо непроективні дерева.

In [12]:
def intersects(n1, n2):
    """Checks whether the spans of two nodes intersect"""
    s1 = n1['id'] if n1['head'] > n1['id'] else n1['head']
    e1 = n1['head'] if n1['head'] > n1['id'] else n1['id']
    s2 = n2['id'] if n2['head'] > n2['id'] else n2['head']
    e2 = n2['head'] if n2['head'] > n2['id'] else n2['id']
    
    return (s1 < s2 and e1 > s2 and e2 > e1) or (s2 < s1 and e2 > s1 and e1 > e2)

def non_projective(tree):
    for n1 in tree:
        for n2 in tree:
            if n1['id'] < n2['id'] and intersects(n1, n2):
                return True
            
    return False

non_projective_trees = list(filter(non_projective, clean_trees))

len(non_projective_trees)

394

Та відфільтровуємо проективні.

In [13]:
projective_trees = list(filter(lambda t: not non_projective(t), clean_trees))

len(projective_trees)

4719

Завантажуємо тестові данні і аналогічнім чином їх фільтруємо.

In [14]:
test_trees = read_treebank('uk_iu-ud-test.conllu')

len(test_trees)

864

In [15]:
test_clean_trees = list(filter(lambda t: not is_broken(t), test_trees))

len(test_clean_trees)

846

In [16]:
test_projective_trees = list(filter(lambda t: not non_projective(t), test_clean_trees))

len(test_projective_trees)

802

In [17]:
test_non_projective_trees = list(filter(non_projective, test_clean_trees))

len(test_non_projective_trees)

44

## Design actions and the oracle

Будуємо класс для роботи зі станом і задаємо операції для swapping arc-standard алгоритму.

In [18]:
from collections import OrderedDict
from enum import Enum

class Actions(str, Enum):
    SHIFT = "shift"
    RIGHT = "right"
    LEFT = "left"
    SWAP = "swap"

class State:
    
    ROOT = OrderedDict([('id', 0), ('form', 'ROOT'), ('lemma', 'ROOT'), ('upostag', 'ROOT'),
                ('xpostag', None), ('feats', None), ('head', None), ('deprel', None),
                ('deps', None), ('misc', None)])
    
    def __init__(self, sent):
        self.stack = [State.ROOT]
        self.queue = sent
        self.relations = []        
        
        # index defines a relative order of a token, during swap the order is changed
        self.stack[-1]['index'] = 0
        self.stack[-1]['head_index'] = None
        
        for i, n in enumerate(self.queue):
            n['index'] = i + 1
            n['head_index'] = n['head']
            
            
    def shift(self):
        self.stack.append(self.queue.pop(0)) 
        
    def left(self):
        self.relations.append((self.stack[-2]["id"], self.stack[-1]["id"]))
        del self.stack[-2]
            
    def right(self):
        self.relations.append((self.stack[-1]["id"], self.stack[-2]["id"]))
        del self.stack[-1]
        
    def swap(self):
        self.queue.insert(0, self.stack[-2])
        tmp = self.stack[-2]['index']
        self.queue[0]['index'] = self.stack[-1]['index']
        self.stack[-1]['index'] = tmp
        for n in self.stack[1:] + self.queue:
            if self.queue[0]['id'] == n['head']:
                n['head_index'] = self.queue[0]['index']

            if self.stack[-1]['id'] == n['head']:
                n['head_index'] = self.stack[-1]['index']

        del self.stack[-2]
        
    def apply(self, action):
        if action == Actions.SHIFT:
            self.shift()
        elif action == Actions.LEFT:
            self.left()
        elif action == Actions.RIGHT:
            self.right()
        elif action == Actions.SWAP:
            self.swap()
        else:
            raise Exception("Unknown action:" + action)
            
    def traverse(self,oracle):
        while len(self.stack) > 1 or self.queue:
            action = oracle(self)
            self.apply(action)
            
            

І будуємо статичний oracle.

In [19]:
def intersects2(n1, n2):
    """Checks whether the spans of two nodes intersect using relative order"""
    s1 = n1['index'] if n1['head_index'] > n1['index'] else n1['head_index']
    e1 = n1['head_index'] if n1['head_index'] > n1['index'] else n1['index']
    s2 = n2['index'] if n2['head_index'] > n2['index'] else n2['head_index']
    e2 = n2['head_index'] if n2['head_index'] > n2['index'] else n2['index']
    
    return (s1 < s2 and e1 > s2 and e2 > e1) or (s2 < s1 and e2 > s1 and e1 > e2)

def is_leaf(node, stack, queue):
    for n in stack:
        if n['head'] == node['id']:
            return False
    for n in queue:
        if n['head'] == node['id']:
            return False    
        
    return True
    
def static_oracle(state):
    stack = state.stack
    queue = state.queue
    if len(stack) > 1:                        
        if stack[-2]['id'] > 0 and stack[-2]['head'] == stack[-1]['id'] and is_leaf(stack[-2], stack, queue):
            return Actions.LEFT
        elif stack[-1]['head'] == stack[-2]['id'] and is_leaf(stack[-1], stack, queue):
            return Actions.RIGHT
        elif stack[-2]['id'] > 0 and stack[-2]['id'] < stack[-1]['id'] and \
        (intersects2(stack[-2], stack[-1]) or \
         [n for n in queue if n['index'] < stack[-2]['head_index'] and intersects2(stack[-2], n)] or\
         [n for n in stack[1:-2] if intersects2(n, stack[-2])]):
            return Actions.SWAP
        else:
            return Actions.SHIFT
    else:
        return Actions.SHIFT

In [20]:
def trace_actions(tree):
    state = State(tree[:])
    def trace(s):
        action = static_oracle(s)
        print("Stack:", [i["form"]+"_"+str(i["id"]) for i in s.stack])
        print("Queue:", [i["form"]+"_"+str(i["id"]) for i in s.queue])
        print("Relations:", s.relations)
        print(action)
        print("========================")
        return action
    
    state.traverse(trace)
    
    print("Gold relations:")
    print([(node["id"], node["head"]) for node in tree])
    print("Retrieved relations:")
    print(sorted(state.relations))

In [21]:
trace_actions(trees[0])

Stack: ['ROOT_0']
Queue: ['У_1', 'домі_2', 'римського_3', 'патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'Венери_11', 'та_12', 'Адоніса_13', '._14']
Relations: []
Actions.SHIFT
Stack: ['ROOT_0', 'У_1']
Queue: ['домі_2', 'римського_3', 'патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'Венери_11', 'та_12', 'Адоніса_13', '._14']
Relations: []
Actions.SHIFT
Stack: ['ROOT_0', 'У_1', 'домі_2']
Queue: ['римського_3', 'патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'Венери_11', 'та_12', 'Адоніса_13', '._14']
Relations: []
Actions.LEFT
Stack: ['ROOT_0', 'домі_2']
Queue: ['римського_3', 'патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', 'Венери_11', 'та_12', 'Адоніса_13', '._14']
Relations: [(1, 2)]
Actions.SHIFT
Stack: ['ROOT_0', 'домі_2', 'римського_3']
Queue: ['патриція_4', 'Руфіна_5', 'була_6', 'прегарна_7', 'фреска_8', ',_9', 'зображення_10', '

## Train a classifier

Визначаємося з фічами.

In [23]:
def extract_state_features(state):
    
    stack, queue, relations = state.stack, state.queue, state.relations
    
    features = dict()
    
    if len(stack) > 1:        
        features["s0-word"] = stack[-2]["form"]
        features["s0-lemma"] = stack[-2]["lemma"]
        features["s0-tag"] = stack[-2]["upostag"]
        features["s0-rchildren-num"] = len([r for r in relations if r[1] == stack[-2]['id']])
        features["s0-lchildren-num"] = len([r for r in relations if r[0] == stack[-2]['id']])
        if stack[-2]["feats"]:
            for k, v in stack[-2]["feats"].items():
                features["s0-" + k] = v
    
    if len(stack) > 2:
        features["s1-word"] = stack[-3]["form"]
        features["s1-tag"] = stack[-3]["upostag"]
    
    if len(stack) > 3:
        features["s2-tag"] = stack[-4]["upostag"]
        
    if len(stack) > 4:
        features["s3-tag"] = stack[-5]["upostag"]
    
    if len(stack) > 1:
        queue_top = stack[-1]
        features["q0-word"] = stack[-1]["form"]
        features["q0-lemma"] = stack[-1]["lemma"]
        features["q0-tag"] = stack[-1]["upostag"]
        features["q0-rchildren-num"] = len([r for r in relations if r[1] == stack[-1]['id']])
        features["q0-lchildren-num"] = len([r for r in relations if r[0] == stack[-1]['id']])
        if stack[-1]["feats"]:
            for k, v in stack[-1]["feats"].items():
                features["q0-" + k] = v
    
    if len(queue) > 0:        
        features["q1-word"] = queue[0]["form"]
        features["q1-tag"] = queue[0]["upostag"]
    
    if len(queue) > 1:
        features["q2-tag"] = queue[1]["upostag"]
    
    if len(queue) > 2:
        features["q3-tag"] = queue[2]["upostag"]
       
    if len(stack) > 1:
        features["distance"] = stack[-1]["id"] - stack[-2]["id"]
    
    features['q-empty'] = not bool(queue)    
    
    return features

def extract_tree_features(tree):
    
    state = State(tree[:])
    
    gold_relations = {(n['id'], n['head']) for n in tree}
    features, labels = [], []
    
    def extract(s):
        action = static_oracle(s)
        labels.append(action.value)
        features.append(extract_state_features(s))
        return action
    
    state.traverse(extract)
    
    #check that gold relatons equal to relations built by oracle
    assert gold_relations == set(state.relations)                  
    return features, labels

def extract_treebank_features(treebank):
    features, labels = [], []
    for tree in treebank:
        tree_features, tree_labels = extract_tree_features(tree)
        features.extend(tree_features)
        labels.extend(tree_labels)
    return features, labels    

Будуємо та тренуємо модель.

In [24]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

class ParseState(State):
    """Special case of state class to handle wrong shift operation"""
    def shift(self):
        if self.queue:
            self.stack.append(self.queue.pop(0)) 
        else:
            #In case of wrong prediction of shift we will get an exception so we need to resolve the situation
            self.left()
    
class DepModel:
    def __init__(self, train_treebank, test_treebank):
        self.train_features, self.train_labels = extract_treebank_features(train_treebank)
        self.test_features, self.test_labels = extract_treebank_features(test_treebank)
        self.vectorizer = DictVectorizer()
        self.test_treebank = test_treebank
        
    def vectorize(self):
        vec = self.vectorizer.fit(self.train_features)
        self.train_features_vec = vec.transform(self.train_features)
        self.test_features_vec = vec.transform(self.test_features)
        
    def train(self):
        self.logreg = LogisticRegression(random_state=26, solver="sag", \
                                         multi_class="multinomial", \
                                         n_jobs = 6, max_iter=3000, verbose=1)
        self.logreg.fit(self.train_features_vec, self.train_labels)
        
    def show_action_metrics(self):
        """
        Prints actions classification F1 metrics
        """
        predicted = self.logreg.predict(self.test_features_vec)
        print(classification_report(self.test_labels, predicted))
        
    def dep_parse(self, tree):
        state = ParseState(tree[:])
        def predict(s):
            features = extract_state_features(state)
            action = self.logreg.predict(self.vectorizer.transform([features]))[0]
            return action
        
        state.traverse(predict)
        return state.relations
        
    def show_UAS_metric(self):
        """
        Calculates and prints unlabeled attachment score
        """
        total, tp = 0, 0
        for tree in self.test_treebank:    
            golden = [(node["id"], node["head"]) for node in tree]
            predicted = self.dep_parse(tree)
            total += len(tree)
            tp += len(set(golden).intersection(set(predicted)))

        print("Total:", total)
        print("Correctly defined:", tp)
        print("UAS:", round(tp/total, 2))
        

Спочатку перевіряємо роботу моделі на проективних деревах.

In [25]:
projective_mod = DepModel(projective_trees, test_projective_trees)

In [26]:
projective_mod.vectorize()
projective_mod.train()

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.


convergence after 1169 epochs took 221 seconds


[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:  3.7min finished


In [27]:
projective_mod.show_action_metrics()

              precision    recall  f1-score   support

        left       0.91      0.91      0.91      7294
       right       0.84      0.79      0.81      7185
       shift       0.87      0.89      0.88     14479

   micro avg       0.87      0.87      0.87     28958
   macro avg       0.87      0.87      0.87     28958
weighted avg       0.87      0.87      0.87     28958



In [28]:
projective_mod.show_UAS_metric()

Total: 14479
Correctly defined: 10403
UAS: 0.72


Оскільки непроективних дерев в тренувальних данних не дуже багато, тому не має великого сенсу тренувати на них модель окремо, тому
ми зразу тренуємо і перевіряємо модель на всіх деревах.

In [29]:
all_mod = DepModel(clean_trees, test_clean_trees)

In [30]:
all_mod.vectorize()
all_mod.train()

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.


convergence after 1773 epochs took 479 seconds


[Parallel(n_jobs=6)]: Done   1 out of   1 | elapsed:  8.0min finished


In [31]:
all_mod.show_action_metrics()

              precision    recall  f1-score   support

        left       0.89      0.91      0.90      8023
       right       0.82      0.77      0.80      7751
       shift       0.85      0.89      0.87     16445
        swap       0.66      0.25      0.36       671

   micro avg       0.85      0.85      0.85     32890
   macro avg       0.81      0.71      0.73     32890
weighted avg       0.85      0.85      0.85     32890



In [32]:
all_mod.show_UAS_metric()

Total: 15774
Correctly defined: 11343
UAS: 0.72


Бачимо що обробка непроективних дерев не підвищила якість UAS, завдяки тому що модель погано передбачає swap операцію. Думаю одна з головних причин це недостатня їх кількість в тренувальних данних для того щом модель мала можливість навчитись.

# Build a parser

Тепер будуємо парсер, який буде використовувати натреновану модель та тестуємо на довільних реченнях, яких немає в тестових данних.

In [66]:
from tokenize_uk import tokenize_uk
import pymorphy2
# pip install russian-tagsets
from russian_tagsets import converters

converter = converters.converter('opencorpora-int', 'ud20')

morph = pymorphy2.MorphAnalyzer(lang='uk')

DET = ['інакший', 'його', 'тамтой', 'чий', 'їх', 'інш.', 'деякий', 'ввесь', 'ваш', 
       'ніякий', 'весь', 'інший', 'чийсь', 'жадний', 'другий', 'кожний', 
       'такий', 'оцей', 'скілька', 'цей', 'жодний', 'все', 'кілька', 'увесь', 
       'кожній', 'те', 'сей', 'ін.', 'отакий', 'котрий', 'усякий', 'самий', 
       'наш', 'усілякий', 'будь-який', 'сам', 'свій', 'всілякий', 'всенький', 'її', 
       'всякий', 'отой', 'небагато', 'який', 'їхній', 'той', 'якийсь', 'ин.', 'котрийсь', 
       'твій', 'мій', 'це']

PREP = ["до", "на"]

mapping = {"ADJF": "ADJ", "ADJS": "ADJ", "COMP": "ADJ", "PRTF": "ADJ",
           "PRTS": "ADJ", "GRND": "VERB", "NUMR": "NUM", "ADVB": "ADV",
           "NPRO": "PRON", "PRED": "ADV", "PREP": "ADP", "PRCL": "PART"}

def normalize_pos(tag, word):
    if word.normal_form in PREP:
        return "PREP"
    elif word.normal_form in DET:
        return "DET"
    else:
        return tag
    
def enrich_with_features(i, word):
    features = dict()
    features['id'] = i
    features['head'] = None
    features['form']  = word
    p = morph.parse(word)[0]
    features['lemma'] = p.normal_form
    pos_feats = list(converter(p.tag._str).split())
    features['upostag'] = normalize_pos(pos_feats[0],p)
    feats = pos_feats[1].split('|') if len(pos_feats) > 1 and pos_feats[1] != '_' else []
    features['feats'] = dict([f.split('=') for f in feats])
    return features

def parse_deps(sent, log = False):
    words = []
    for i, word in enumerate(tokenize_uk.tokenize_words(sent)):
        words.append(enrich_with_features(i+1, word))
        
    if log:    
        print(words)
             
    relations = all_mod.dep_parse(words)
    word_by_id = dict()
    
    for word in words:
        word_by_id[word['id']] = word
    
    for rel in relations:
        word_by_id[rel[0]]['head'] = rel[1]
    
    return words
    

Приклад простого речення, майже всі зв'язки правильні.

In [67]:
print_rels(parse_deps('Тепер будуємо парсер, який буде використовувати натреновану модель'))

Тепер <-- будуємо
будуємо <-- root
парсер <-- будуємо
, <-- буде
який <-- буде
буде <-- парсер
використовувати <-- буде
натреновану <-- використовувати
модель <-- будуємо


Намагаємось розпарсити складне речення. Є також багто неправильних зв'яків. Можна бачити я пропагується помилка, наприклад замість 'величиною' у багатьох слов батьком повинен був бути 'місті'.

In [68]:
print_rels(parse_deps('У Гетеборзі, другому за величиною місті Швеції, та в Копенгагені, столиці Данії, на першотравневих мітингах сталися сутички між поліцією та протестувальниками.'))

У <-- Гетеборзі
Гетеборзі <-- величиною
, <-- величиною
другому <-- величиною
за <-- величиною
величиною <-- root
місті <-- Швеції
Швеції <-- величиною
, <-- Копенгагені
та <-- Копенгагені
в <-- Копенгагені
Копенгагені <-- величиною
, <-- столиці
столиці <-- Копенгагені
Данії <-- столиці
, <-- сталися
на <-- сталися
першотравневих <-- мітингах
мітингах <-- сталися
сталися <-- величиною
сутички <-- сталися
між <-- поліцією
поліцією <-- сутички
та <-- протестувальниками
протестувальниками <-- поліцією
. <-- величиною


Ще один приклад простого речення, здається все правильно.

In [69]:
print_rels(parse_deps('Зеленський і Система: які варіанти розвитку подій.'))

Зеленський <-- Система
і <-- Система
Система <-- root
: <-- варіанти
які <-- варіанти
варіанти <-- Система
розвитку <-- варіанти
подій <-- розвитку
. <-- Система


Приклад непроективного речення, оброблюється некорекнто.

In [70]:
print_rels(parse_deps('Василь бачив собаку вчора породи спаніель.'))

Василь <-- бачив
бачив <-- root
собаку <-- бачив
вчора <-- собаку
породи <-- спаніель
спаніель <-- вчора
. <-- бачив
